<a href="https://colab.research.google.com/github/MarMarhoun/freelance_work/blob/main/side_projects/NLP_projs/LLMs_with_Gradio/image_seg_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medical Image Segmentation: for X-ray and MRI Analysis

To create an advanced image segmentation application using Gradio and Large Language Models (LLMs) from Hugging Face, we can follow these steps. The application will take an input image (such as an X-ray or MRI scan) from a URL, use a model to generate a segmentation mask, and then display both the mask and the segmented image.

* Section 1: Medical Image Segmentation with LLMs
* Section 2: Dynamic Medical Image Segmentation with LLMs




In [51]:
# Install the necessary libraries
!pip install gradio torch torchvision

# Section 1: Medical Image Segmentation with LLMs

In [52]:
import gradio as gr
import requests
from io import BytesIO
from PIL import Image
import numpy as np
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration

# Load the BLIP model and processor from Hugging Face for image captioning
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Define a fixed color palette for five regions
color_palette = [
    (0, 0, 0),    # Red
    (0, 255, 0),    # Green
    (0, 0, 255),    # Blue
    (255, 255, 0),  # Yellow
    (255, 165, 0)   # Orange
]

def segment_image(image):
    # Convert the image to grayscale
    gray_image = image.convert("L")
    gray_array = np.array(gray_image)

    # Create an empty array for the colored segmented image
    colored_image = np.zeros((*gray_array.shape, 3), dtype=np.uint8)

    # Define the thresholds for the five regions
    thresholds = np.linspace(0, 255, num=6).astype(int)  # 5 regions

    # Assign colors based on grayscale values
    for i in range(5):
        mask = (gray_array >= thresholds[i]) & (gray_array < thresholds[i + 1])
        colored_image[mask] = color_palette[i]

    # Convert the colored image back to PIL format
    colored_image_pil = Image.fromarray(colored_image)

    return image, gray_image, colored_image_pil  # Return original, grayscale, and colored images

def generate_caption(image):
    # Preprocess the image for the BLIP model
    inputs = processor(images=image, return_tensors="pt")

    # Generate a caption
    with torch.no_grad():
        output = model.generate(**inputs)

    # Decode the generated caption
    caption = processor.decode(output[0], skip_special_tokens=True)
    return caption

def update_image(image_url):
    # This function fetches the image using request and returns it
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content)).convert("RGB")
    return image

# Create the Gradio interface
with gr.Blocks() as iface:
    gr.Markdown("<h1 style='text-align: center;'>Medical Image Segmentation with LLMs</h1>")
    gr.Markdown("Enter the URL of an X-ray or MRI image to generate a segmentation mask and the segmented image. \n\n"
                "Example URLs:\n"
                "- X-ray: https://as2.ftcdn.net/v2/jpg/02/63/69/43/1000_F_263694305_nxVMuaTWhqa7uPAT0iUd5BaTzA6UU467.jpg \n"
                "- X-ray: https://as2.ftcdn.net/v2/jpg/01/97/03/19/1000_F_197031952_MGjgPGxw58GG6vQvA8xZh2f015WISGPe.jpg \n"
                "- X-ray: https://as1.ftcdn.net/v2/jpg/00/35/87/38/1000_F_35873899_xDcthI9k5yHGoL6B4jAyN5s1l2Ra3Weq.jpg \n"
                "- X-ray: https://as1.ftcdn.net/v2/jpg/02/66/34/72/1000_F_266347240_LmEx8algXY1sRE9stHPPtqJbGKKDtvh4.jpg \n"
                "- X-ray: https://as1.ftcdn.net/v2/jpg/04/19/34/16/1000_F_419341614_sFC7I0D9I5JSTeKwUcwAfNGzgEb3gdHO.jpg \n"
                "- MRI: https://www.kenhub.com/thumbor/zoz_XVCq44UFroH2ds6eoOUvdtA=/fit-in/800x1600/filters:watermark(/images/logo_url.png,-10,-10,0):background_color(FFFFFF):format(jpeg)/images/library/13517/ff.jpg \n"
                "- MRI: https://upload.wikimedia.org/wikipedia/commons/thumb/b/b2/MRI_of_Human_Brain.jpg/330px-MRI_of_Human_Brain.jpg \n"
                "- MRI: https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Normal_axial_T2-weighted_MR_image_of_the_brain.jpg/250px-Normal_axial_T2-weighted_MR_image_of_the_brain.jpg \n"
                "- MRI: https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/Atelectasia1.jpg/220px-Atelectasia1.jpg"

                )

    # Input for image URL
    with gr.Row():
        with gr.Column():
            # Input for image URL
            image_url = gr.Textbox(label="Image URL", placeholder="Enter the URL of the image")
            # Button for uploading the image
            upload_button = gr.Button("Upload Image")
            # Call update_image function, which fetches and returns the image
            upload_button.click(fn=update_image, inputs=image_url, outputs=uploaded_image)
        with gr.Column():
            # Output for the fetched image
            uploaded_image = gr.Image(label="Uploaded Image", type="pil")
            upload_button.click(fn=update_image, inputs=image_url, outputs=uploaded_image)


    # Row for original, grayscale, and colored segmented images
    with gr.Row():
        original_image_output = gr.Image(label="Original Image")
        gray_image_output = gr.Image(label="Grayscale Image")
        colored_image_output = gr.Image(label="Colored Segmented Image")


    with gr.Row():
      run_button = gr.Button("Run Segmentation Test")
    # Define the action on button click for the run button
      run_button.click(segment_image, inputs=uploaded_image, outputs=[original_image_output, gray_image_output, colored_image_output])

# Launch the Gradio app
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e754ef7c7dd6fbc9f0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Dynamic Medical Image Segmentation with LLMs

In [53]:
import gradio as gr
import requests
from io import BytesIO
from PIL import Image
import numpy as np
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration

# Load the BLIP model and processor from Hugging Face for image captioning
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Define a fixed color palette for regions
color_palette = [
    (0, 0, 0),    # Red
    (0, 255, 0),    # Green
    (0, 0, 255),    # Blue
    (255, 255, 0),  # Yellow
    (255, 165, 0)   # Orange
]

def segment_image(image, num_regions):
    # Convert the image to grayscale
    gray_image = image.convert("L")
    gray_array = np.array(gray_image)

    # Create an empty array for the colored segmented image
    colored_image = np.zeros((*gray_array.shape, 3), dtype=np.uint8)

    # Define the thresholds for the specified number of regions
    thresholds = np.linspace(0, 255, num=num_regions + 1).astype(int)  # num_regions + 1 for boundaries

    # Assign colors based on grayscale values
    for i in range(num_regions):
        mask = (gray_array >= thresholds[i]) & (gray_array < thresholds[i + 1])
        colored_image[mask] = color_palette[i % len(color_palette)]  # Cycle through colors if more regions than colors

    # Convert the colored image back to PIL format
    colored_image_pil = Image.fromarray(colored_image)

    return image, gray_image, colored_image_pil  # Return original, grayscale, and colored images

def generate_caption(image):
    # Preprocess the image for the BLIP model
    inputs = processor(images=image, return_tensors="pt")

    # Generate a caption
    with torch.no_grad():
        output = model.generate(**inputs)

    # Decode the generated caption
    caption = processor.decode(output[0], skip_special_tokens=True)
    return caption

def update_image(image_url):
    # This function fetches the image using request and returns it
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content)).convert("RGB")
    return image

# Create the Gradio interface
with gr.Blocks() as iface:
    gr.Markdown("<h1 style='text-align: center;'>Dynamic Medical Image Segmentation with LLMs</h1>")
    gr.Markdown("Enter the URL of an X-ray or MRI image to generate a segmentation mask and the segmented image. \n\n"
                "Example URLs:\n"
                "- X-ray: https://as2.ftcdn.net/v2/jpg/02/63/69/43/1000_F_263694305_nxVMuaTWhqa7uPAT0iUd5BaTzA6UU467.jpg \n"
                "- X-ray: https://as2.ftcdn.net/v2/jpg/01/97/03/19/1000_F_197031952_MGjgPGxw58GG6vQvA8xZh2f015WISGPe.jpg \n"
                "- X-ray: https://as1.ftcdn.net/v2/jpg/00/35/87/38/1000_F_35873899_xDcthI9k5yHGoL6B4jAyN5s1l2Ra3Weq.jpg \n"
                "- X-ray: https://as1.ftcdn.net/v2/jpg/02/66/34/72/1000_F_266347240_LmEx8algXY1sRE9stHPPtqJbGKKDtvh4.jpg \n"
                "- X-ray: https://as1.ftcdn.net/v2/jpg/04/19/34/16/1000_F_419341614_sFC7I0D9I5JSTeKwUcwAfNGzgEb3gdHO.jpg \n"
                "- MRI: https://www.kenhub.com/thumbor/zoz_XVCq44UFroH2ds6eoOUvdtA=/fit-in/800x1600/filters:watermark(/images/logo_url.png,-10,-10,0):background_color(FFFFFF):format(jpeg)/images/library/13517/ff.jpg \n"
                "- MRI: https://upload.wikimedia.org/wikipedia/commons/thumb/b/b2/MRI_of_Human_Brain.jpg/330px-MRI_of_Human_Brain.jpg \n"
                "- MRI: https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Normal_axial_T2-weighted_MR_image_of_the_brain.jpg/250px-Normal_axial_T2-weighted_MR_image_of_the_brain.jpg \n"
                "- MRI: https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/Atelectasia1.jpg/220px-Atelectasia1.jpg"

                )
    # Input for image URL
    with gr.Row():
        with gr.Column():
            # Input for image URL
            image_url = gr.Textbox(label="Image URL", placeholder="Enter the URL of the image")
            # Button for uploading the image
            upload_button = gr.Button("Upload Image")
            # Call update_image function, which fetches and returns the image
            upload_button.click(fn=update_image, inputs=image_url, outputs=uploaded_image)
            # Slider for number of regions
            num_regions_slider = gr.Slider(minimum=2, maximum=20, step=1, label="Number of Regions", value=5)

        with gr.Column():
            # Output for the fetched image
            uploaded_image = gr.Image(label="Uploaded Image", type="pil")
            upload_button.click(fn=update_image, inputs=image_url, outputs=uploaded_image)



    # Row for original, grayscale, and colored segmented images
    with gr.Row():
        original_image_output = gr.Image(label="Original Image")
        gray_image_output = gr.Image(label="Grayscale Image")
        colored_image_output = gr.Image(label="Colored Segmented Image")


    with gr.Row():
        run_button = gr.Button("Run Segmentation Test")
        # Define the action on button click for the run button
        run_button.click(segment_image, inputs=[uploaded_image, num_regions_slider], outputs=[original_image_output, gray_image_output, colored_image_output])

# Launch the Gradio app
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f86dffa5429aee4756.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
